### Importar librerías y Cargar Datos

In [22]:
import numpy as np
import pandas as pd
import os

# ---------------------------
# CARGA DE ARCHIVO
# ---------------------------
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPC1C2.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPC1C2.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/FPC1C2.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):
        df = pd.read_excel(path, header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")
df.head()

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPC1C2.xlsx


,Unnamed: 0,SERIE,FECHA,C1 H1 %,C1 H2 %,C1 H3 %,C1 H0 %,C1 H1 pF,C1 H2 pF,C1 H3 pF,...,C2 H1 %,C2 H2 %,C2 H3 %,C2 H0 %,C2 H1 pF,C2 H2 pF,C2 H3 pF,C2 H0 pF,TempR,HumR
0,NaN,59571,2016-10-23,0.34,NaN,NaN,0.37,280.25,NaN,NaN,...,0.21,NaN,NaN,6.33,1114.5,NaN,NaN,373.17,NaN,NaN
1,NaN,59571,2006-01-29,0.36,NaN,NaN,0.40,281.20,NaN,NaN,...,0.19,NaN,NaN,0.25,1053.0,NaN,NaN,360.60,NaN,NaN
2,NaN,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,59572,2006-01-29,0.37,NaN,NaN,0.34,288.10,NaN,NaN,...,0.21,NaN,NaN,0.26,1072.0,NaN,NaN,353.60,NaN,NaN


### Limpieza de datos

In [23]:
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------
df = df.drop(columns=df.columns[0])
# df["SERIE"] = df["SERIE"].astype(str)
cols_base = ['SERIE', 'FECHA']

# quedarnos solo con columnas de capacitancia (pF)
cols_c = cols_base + [col for col in df.columns if col.endswith('pF')]
df_c = df[cols_c].copy()
df_c.head()

,SERIE,FECHA,C1 H1 pF,C1 H2 pF,C1 H3 pF,C1 H0 pF,C2 H1 pF,C2 H2 pF,C2 H3 pF,C2 H0 pF
0,59571,2016-10-23,280.25,NaN,NaN,190.91,1114.5,NaN,NaN,373.17
1,59571,2006-01-29,281.20,NaN,NaN,191.80,1053.0,NaN,NaN,360.60
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,288.10,NaN,NaN,192.50,1072.0,NaN,NaN,353.60


### SubTabla C1

In [24]:
# ---------------------------
# SUBTABLA C1
# ---------------------------
cols_c_c1 = [col for col in df_c.columns if col.startswith('C1')]
df_c_c1 = df_c[cols_base + cols_c_c1].copy()

# FECHA a datetime
df_c_c1["FECHA"] = pd.to_datetime(df_c_c1["FECHA"])

# calcular % de variación respecto al valor inicial por SERIE
cap_cols = [c for c in df_c_c1.columns if c.endswith("pF")]
for col in cap_cols:
    ref = df_c_c1.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
    ref_mapped = df_c_c1["SERIE"].map(ref)
    delta = abs((df_c_c1[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)
    mask = df_c_c1[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df_c_c1[col] == 0)
    delta[mask] = np.nan
    df_c_c1[f"{col}_Delta"] = delta

df_c_c1.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_21068\2025396333.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df_c_c1.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_21068\2025396333.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df_c_c1.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppDa

,SERIE,FECHA,C1 H1 pF,C1 H2 pF,C1 H3 pF,C1 H0 pF,C1 H1 pF_Delta,C1 H2 pF_Delta,C1 H3 pF_Delta,C1 H0 pF_Delta
0,59571,2016-10-23,280.25,NaN,NaN,190.91,NaN,NaN,NaN,NaN
1,59571,2006-01-29,281.20,NaN,NaN,191.80,NaN,NaN,NaN,NaN
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,288.10,NaN,NaN,192.50,NaN,NaN,NaN,NaN


In [25]:
# máximo de variaciones
cols_c_c1_delta = [col for col in df_c_c1.columns if col.endswith('Delta')]
df_c_c1["Max_C_C1"] = df_c_c1[cols_c_c1_delta].max(axis=1)

In [26]:
df_c_c1.head()

,SERIE,FECHA,C1 H1 pF,C1 H2 pF,C1 H3 pF,C1 H0 pF,C1 H1 pF_Delta,C1 H2 pF_Delta,C1 H3 pF_Delta,C1 H0 pF_Delta,Max_C_C1
0,59571,2016-10-23,280.25,NaN,NaN,190.91,NaN,NaN,NaN,NaN,NaN
1,59571,2006-01-29,281.20,NaN,NaN,191.80,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,288.10,NaN,NaN,192.50,NaN,NaN,NaN,NaN,NaN


### Puntaje para C1

In [27]:
# ---------------------------
# CONFIGURACIÓN DE PUNTAJE
# ---------------------------
pesos = [5, 3, 1]  
limites = {"C": [10, 5]}  # [límite superior, límite inferior]

def asignar_puntaje(df, columna_valor, columna_puntaje, limites, pesos):
    df[columna_puntaje] = np.select(
        [
            df[columna_valor] > limites[0],
            (df[columna_valor] > limites[1]) & (df[columna_valor] <= limites[0]),
            df[columna_valor] <= limites[1]
        ],
        pesos,
        default=np.nan
    )
    return df

# aplicar reglas
df_c_c1 = asignar_puntaje(df_c_c1, "Max_C_C1", "CBC1", limites["C"], pesos)
df_c_c1.head()


,SERIE,FECHA,C1 H1 pF,C1 H2 pF,C1 H3 pF,C1 H0 pF,C1 H1 pF_Delta,C1 H2 pF_Delta,C1 H3 pF_Delta,C1 H0 pF_Delta,Max_C_C1,CBC1
0,59571,2016-10-23,280.25,NaN,NaN,190.91,NaN,NaN,NaN,NaN,NaN,NaN
1,59571,2006-01-29,281.20,NaN,NaN,191.80,NaN,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,288.10,NaN,NaN,192.50,NaN,NaN,NaN,NaN,NaN,NaN


### Tabla Base es la tabla con columnas "SERIE","FECHA","CBC1"

In [28]:
# ---------------------------
# TABLAS BASE
# ---------------------------
df_C_C1 = df_c_c1[["SERIE", "FECHA", "CBC1"]].copy()  # tabla original
df_full_C1 = df_c_c1[["SERIE", "FECHA", "CBC1"] + cap_cols].copy()  # tabla detallada
df_full_C1.head()

,SERIE,FECHA,CBC1,C1 H1 pF,C1 H2 pF,C1 H3 pF,C1 H0 pF
0,59571,2016-10-23,NaN,280.25,NaN,NaN,190.91
1,59571,2006-01-29,NaN,281.20,NaN,NaN,191.80
2,59571,1996-04-16,NaN,NaN,NaN,NaN,NaN
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,NaN,288.10,NaN,NaN,192.50


### Extensión de calendario desde 2015 al dia de hoy

In [29]:
# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2015
# ---------------------------
inicio = "2015-01-01"
fecha_inicio = pd.Timestamp(inicio)
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

todas_series = df["SERIE"].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- tabla extendida CBC1 ----------
ultimos_2015 = df_C_C1[df_C_C1["FECHA"] < fecha_inicio].sort_values("FECHA").groupby("SERIE").tail(1)
ultimos_2015["FECHA"] = fecha_inicio
base_ext = pd.concat([df_C_C1, ultimos_2015], ignore_index=True)

df_extendida_C_C1 = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida_C_C1 = df_extendida_C_C1.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- tabla detallada extendida ----------
ultimos_2015_det = df_full_C1[df_full_C1["FECHA"] < fecha_inicio].sort_values("FECHA").groupby("SERIE").tail(1)
ultimos_2015_det["FECHA"] = fecha_inicio
base_ext_det = pd.concat([df_full_C1, ultimos_2015_det], ignore_index=True)

df_detalles_ext_C_C1 = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_detalles_ext_C_C1 = df_detalles_ext_C_C1.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)


C:\Users\roquispec\AppData\Local\Temp\ipykernel_21068\2319882772.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_C_C1 = df_extendida_C_C1.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_21068\2319882772.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_detalles_ext_C_C1 = df_detalles_ext_C_C1.groupby("SERIE").apply(lambda g: g.

### Funciones de llamada

In [30]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_C_C1():
    return df_C_C1

def get_df_extendida_C_C1():
    return df_extendida_C_C1

def get_df_detalles_C_C1():
    return df_full_C1

def get_df_detalles_ext_C_C1():
    return df_detalles_ext_C_C1

# ---------------------------
# DEMO DE RESULTADOS
# ---------------------------
print("\n ====== TABLA CBC1 ORIGINAL ====== \n")
print(get_df_C_C1(), "\n")

print("\n ====== TABLA CBC1 EXTENDIDA ====== \n")
print(get_df_extendida_C_C1().head(), "\n")

print("\n ====== TABLA DETALLADA CBC1 ====== \n")
print(get_df_detalles_C_C1().head(), "\n")

print("\n ====== TABLA DETALLADA EXTENDIDA CBC1 ====== \n")
print(get_df_detalles_ext_C_C1().tail(), "\n")


 ====== TABLA CBC1 ORIGINAL ====== 

            SERIE      FECHA  CBC1
0           59571 2016-10-23   NaN
1           59571 2006-01-29   NaN
2           59571 1996-04-16   NaN
3           59572 2016-09-18   NaN
4           59572 2006-01-29   NaN
..            ...        ...   ...
383  5337PA196-03 2025-01-19   NaN
384       D518294 2025-06-05   NaN
385       D518294 2025-06-05   NaN
386        359111 2022-12-14   NaN
387       L-30230 2023-11-02   NaN

[388 rows x 3 columns] 


 ====== TABLA CBC1 EXTENDIDA ====== 

    SERIE      FECHA  CBC1
0  100138 2015-01-01   NaN
1  100138 2015-01-02   NaN
2  100138 2015-01-03   NaN
3  100138 2015-01-04   NaN
4  100138 2015-01-05   NaN 


 ====== TABLA DETALLADA CBC1 ====== 

   SERIE      FECHA  CBC1  C1 H1 pF  C1 H2 pF  C1 H3 pF  C1 H0 pF
0  59571 2016-10-23   NaN    280.25       NaN       NaN    190.91
1  59571 2006-01-29   NaN    281.20       NaN       NaN    191.80
2  59571 1996-04-16   NaN       NaN       NaN       NaN       NaN
3  59572 2